Firstly, import everything needed.

In [1]:
import pandas as pd

from typing import List

Here I created a function to fast take the data from the files.

In [2]:
def read_data(__file_name: str, __col_names: List[str], path: str = "../data/raw/ml-100k", **kwargs):
    """
    Reads the data from the given file.
    Optimized for using for the main data (ml-100k).

    Parameters:
        __file_name: Name of the file with data.
        __col_names: Names of the columns in the dataset.
        path: Path of the file.
    
    Returns:
        pd.DataFrame.
    """
    return pd.read_csv(f"{path}/{__file_name}", names=__col_names, **kwargs)

Main dataset—users and their ratings to some movie.

In [3]:
main_dataset = read_data("u.data", ["UserId", "ItemId", "Rating", "Timestamp"], sep="\t")
main_dataset.head()

,UserId,ItemId,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Let's look at this dataset more carefully.

In [4]:
main_dataset.isna().sum()

UserId       0
ItemId       0
Rating       0
Timestamp    0
dtype: int64

In [5]:
len(main_dataset)

100000

In [6]:
main_dataset.nunique()

UserId         943
ItemId        1682
Rating           5
Timestamp    49282
dtype: int64

They're a lot of unique values in the `Timestamp` column.
Let's check one more thing: are the same value related to the same person.

In [7]:
main_dataset.sort_values("Timestamp")

,UserId,ItemId,Rating,Timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843
...,...,...,...,...
46773,729,689,4,893286638
73008,729,313,3,893286638
46574,729,328,3,893286638
64312,729,748,4,893286638


Even if there are some timestamps duplicate each other, they are related to the same user.
To suggest any movies we don't need this information, thus, it would be better to delete it.

In [8]:
main_dataset.drop("Timestamp", axis=1, inplace=True)

In [9]:
main_dataset.describe()

,UserId,ItemId,Rating
count,100000.00000,100000.000000,100000.000000
mean,462.48475,425.530130,3.529860
std,266.61442,330.798356,1.125674
min,1.00000,1.000000,1.000000
25%,254.00000,175.000000,3.000000
50%,447.00000,322.000000,4.000000
75%,682.00000,631.000000,4.000000
max,943.00000,1682.000000,5.000000


Before I'll upload the movies, I'm uploading the list of genres of the movies, thus, I'll use them to take the full data from the next file.

In [10]:
genres = read_data("u.genre", ["Name", "Id"], sep="|")
genres.head()

,Name,Id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


And now the dataset of the movies.

In [11]:
movies_dataset_names = ["Id", "Title", "ReleaseDate", "VideoReleaseDate", "IMDB_URL"] + genres.Name.tolist()
movies = read_data("u.item", movies_dataset_names, sep="|", encoding="latin-1")
movies.head()

,Id,Title,ReleaseDate,VideoReleaseDate,IMDB_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Let's analyze it.

In [12]:
movies.isna().sum()

Id                     0
Title                  0
ReleaseDate            1
VideoReleaseDate    1682
IMDB_URL               3
unknown                0
Action                 0
Adventure              0
Animation              0
Children's             0
Comedy                 0
Crime                  0
Documentary            0
Drama                  0
Fantasy                0
Film-Noir              0
Horror                 0
Musical                0
Mystery                0
Romance                0
Sci-Fi                 0
Thriller               0
War                    0
Western                0
dtype: int64

I have no idea why we need information about when the movie's video was released; it will not help us anyhow suggest the films in real life.
So I'll remove this column to exclude the problem with multiple `Nan`s in the dataset.

In [13]:
movies.drop("VideoReleaseDate", axis=1, inplace=True)

`Release date` may help us to suggest films from one era.
To fill this `Nan` value I'll just google the release date of this film.

In [14]:
movies[movies["ReleaseDate"].isna()]

,Id,Title,ReleaseDate,IMDB_URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
266,267,unknown,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ok, it's just something unusual.
We can completely remove this movie from the dataset, because we even cannot just it to somebody.

In [15]:
movies.dropna(subset=["ReleaseDate"], inplace=True)

It's also needed to remove it from the dataset with ratings.

In [16]:
main_dataset.drop(main_dataset[main_dataset["ItemId"] == 267].index, inplace=True)

The last one, let's check the `Nan`s in the `IMDB` column.

In [17]:
movies[movies["IMDB_URL"].isna()]

,Id,Title,ReleaseDate,IMDB_URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1357,1358,The Deadly Cure (1996),16-Sep-1996,NaN,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1358,1359,Boys in Venice (1996),24-Sep-1996,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I googled and found the first movie: [The Deadly Cure](https://www.imdb.com/title/tt0645145/).

In [18]:
movies[movies["IMDB_URL"] == "The Deadly Cure"] = "https://www.imdb.com/title/tt0645145/"

But there is a problem with another: `Boys in Venice` is not googled.
It exists with 1978 year released. I have to find the film with genre "Drama",
but this [movie](https://www.imdb.com/title/tt0211986/) has another genre, and I'm not sure that's what people want to see as suggestions for Drama.
There is no another option on the internet, so I will just print every time that there are no any links to this film.

In [19]:
movies.fillna({"IMDB_URL": "no links available"}, inplace=True)

In [20]:
movies.isna().sum()

Id             0
Title          0
ReleaseDate    0
IMDB_URL       0
unknown        0
Action         0
Adventure      0
Animation      0
Children's     0
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
dtype: int64

Perfect. Now let's check uniqueness

In [21]:
movies.nunique()

Id             1681
Title          1663
ReleaseDate     240
IMDB_URL       1661
unknown           2
Action            2
Adventure         2
Animation         2
Children's        2
Comedy            2
Crime             2
Documentary       2
Drama             2
Fantasy           2
Film-Noir         2
Horror            2
Musical           2
Mystery           2
Romance           2
Sci-Fi            2
Thriller          2
War               2
Western           2
dtype: int64

It's not right the `Title` column does not have all distinct values, some rows contain duplicates. Let's look at them.

In [22]:
movies[movies["Title"].duplicated()]

,Id,Title,ReleaseDate,IMDB_URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
267,268,Chasing Amy (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?Chasing+Amy+(...,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
302,303,Ulee's Gold (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?Ulee%27s+Gold...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
347,348,Desperate Measures (1998),30-Jan-1998,http://us.imdb.com/Title?Desperate+Measures+(1...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
499,500,Fly Away Home (1996),13-Sep-1996,http://us.imdb.com/M/title-exact?Fly%20Away%20...,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
669,670,Body Snatchers (1993),01-Jan-1993,http://us.imdb.com/M/title-exact?Body%20Snatch...,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
679,680,Kull the Conqueror (1997),29-Aug-1997,http://us.imdb.com/M/title-exact?Kull+the+Conq...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
864,865,"Ice Storm, The (1997)",01-Jan-1997,http://us.imdb.com/M/title-exact?Ice+Storm%2C+...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
880,881,Money Talks (1997),22-Aug-1997,http://us.imdb.com/M/title-exact?Money+Talks+(...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1002,1003,That Darn Cat! (1997),14-Feb-1997,http://us.imdb.com/M/title-exact?That%20Darn%2...,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1256,1257,"Designated Mourner, The (1997)",23-May-1997,http://us.imdb.com/M/title-exact?Designated%20...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
movies[movies["Title"] == "Chasing Amy (1997)"]

,Id,Title,ReleaseDate,IMDB_URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
245,246,Chasing Amy (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?Chasing+Amy+(...,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
267,268,Chasing Amy (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?Chasing+Amy+(...,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


These two rows are the same.
So I guess we don't need that, so we can remove all duplicates changing ids in the main dataset.

In [24]:
def remove_movie_duplicates(__main_df: pd.DataFrame, __movies_df: pd.DataFrame):
    """
    Removes duplicates of movies in the `movies` and main datasets.
    The datasets are changed in place.

    Parameters:
        __main_df: Main dataset - dataset of ratings of every movie by every user.
        __movies_df: Dataset of movies - their ids, titles, links, etc.
    """
    for _title in __movies_df[__movies_df["Title"].duplicated()]["Title"].to_list():
        _main_id, _duplicated_id = __movies_df[__movies_df["Title"] == _title]["Id"].to_list()
        __main_df.drop(__main_df[__main_df["ItemId"] == _duplicated_id].index, inplace=True)
    __movies_df.drop_duplicates(subset=["Title"], inplace=True)

In [25]:
remove_movie_duplicates(main_dataset, movies)

In [26]:
movies[movies["Title"].duplicated()]

,Id,Title,ReleaseDate,IMDB_URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western


In [27]:
main_dataset[main_dataset.duplicated()]

,UserId,ItemId,Rating


In [28]:
main_dataset[main_dataset["ItemId"] == 268]

,UserId,ItemId,Rating


In [29]:
main_dataset[main_dataset["ItemId"] == 246]

,UserId,ItemId,Rating
30,50,246,3
493,269,246,5
1347,99,246,3
2122,276,246,4
2986,79,246,5
...,...,...,...
90510,778,246,2
93806,928,246,5
94305,486,246,3
97315,243,246,4


In [30]:
main_dataset.nunique()

UserId     943
ItemId    1663
Rating       5
dtype: int64

Good. Now we can move to the users' dataset

In [31]:
users = read_data("u.user", ["Id", "Age", "Gender", "Occupation", "ZipCode"], sep="|")
users.head()

,Id,Age,Gender,Occupation,ZipCode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [32]:
users.isna().sum()

Id            0
Age           0
Gender        0
Occupation    0
ZipCode       0
dtype: int64

In [33]:
users.nunique()

Id            943
Age            61
Gender          2
Occupation     21
ZipCode       795
dtype: int64

In [34]:
len(users)

943

There are some many distinct zip codes of users.
I got an idea that it's possible to convert zip codes to the name of the cities, for example.
But I'm not sure if it'd be nice, because as far as I understand, it does not matter where live to suggest a movie to you.
This is why I made a conclusion deletion of this column is the best choice here.

In [35]:
users.drop("ZipCode", axis=1, inplace=True)

In [36]:
users.duplicated().sum()

0

Let's check if all movies are rated at least once:

In [37]:
main_dataset["ItemId"].nunique()

1663

Wow, 5 movies are never rated. Let's check remove them.

In [46]:
ids = set([i for i in range(1, 1668)])
not_rated_ids = ids.difference(main_dataset["ItemId"].values)
movies.drop(movies["Id"].isin(not_rated_ids).index, inplace=True)

So the last thing I want to do is to prepare the data for the RecSys model.
I'll merge all datasets into one.

In [48]:
movies.rename(columns={"Id": "ItemId"}, inplace=True)
users.rename(columns={"Id": "UserId"}, inplace=True)

In [49]:
df = pd.merge(main_dataset, users, on="UserId", how="left")
df = pd.merge(df, movies, on="ItemId", how="left")

In [50]:
df.head()

,UserId,ItemId,Rating,Age,Gender,Occupation,Title,ReleaseDate,IMDB_URL,unknown,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,49,M,writer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,186,302,3,39,F,executive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22,377,1,25,M,writer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,244,51,2,28,M,technician,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,166,346,1,47,M,educator,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df.to_csv("../data/interim/ratings.csv", index=False)